In [1]:
#importing libraries : Pandas,numpy,string,regular expression,natural language tool kit,Scikit-learn,tensorflow,keras
import pandas as pd
import numpy as np
import string
import re
import nltk
#downloading the punctuations and stopwords from natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
#importing stopwords for the tweets
from nltk.corpus import stopwords

sw = stopwords.words('english')
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
#importing the layers required for the model
from tensorflow.keras.layers import Dense, LSTM,Bidirectional, Dropout,MaxPool2D,Conv1D,MaxPool1D,RNN,SimpleRNN,LSTMCell,TimeDistributed,GRU,GRUCell
#importing the label encoding
from sklearn.preprocessing import LabelEncoder
#importing tokenizers from natural language toolkit
from nltk.tokenize import word_tokenize, sent_tokenize
#importing the embedding layer
from keras.layers import Embedding
#importing metrics
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score

[nltk_data] Downloading package punkt to /home/sanju/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sanju/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
#reading the training dataset
data = pd.read_csv('train.csv')

In [3]:
#reading the training dataset
testdata = pd.read_csv('test.csv')

# New Section

In [4]:
#reading headers in training data
data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
#reading headers in testing data
testdata.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [6]:
#removing the blank spaces
data = data.dropna()

In [7]:
#Assigning the text in the dataset to variable X
X = data['text']

In [8]:
#test variables
X_test=testdata['text']
y_test=testdata['sentiment']
y_test=LabelEncoder().fit_transform(y_test)

In [9]:
#Assigning the sentiment in the dataset to variable Y
y = data['sentiment']

In [10]:
y

0         neutral
1        negative
2        negative
3        negative
4        negative
5         neutral
6        positive
7         neutral
8         neutral
9        positive
10        neutral
11       positive
12       negative
13       negative
14        neutral
15       negative
16       negative
17       negative
18       negative
19        neutral
20        neutral
21       positive
22        neutral
23        neutral
24        neutral
25       positive
26       negative
27       negative
28       positive
29       negative
           ...   
27451    positive
27452     neutral
27453     neutral
27454    positive
27455    positive
27456    negative
27457     neutral
27458    positive
27459    positive
27460     neutral
27461    positive
27462    positive
27463    negative
27464    negative
27465    positive
27466     neutral
27467    positive
27468     neutral
27469     neutral
27470    negative
27471     neutral
27472    negative
27473     neutral
27474    positive
27475    p

In [11]:
#application of label encoding and tranforming y
y = LabelEncoder().fit_transform(y)


In [12]:
y


array([1, 0, 0, ..., 2, 2, 1])

In [13]:
#Coverting the string to lower case and iterating through the words and appending them to the list if the word is not a for X
X = X.str.lower()

lst = []
for phase_word in X:
    lst.append(" ".join([re.sub('[0-9\W_]', '', word) for word in phase_word.split() if not word in sw]))
X = lst

In [14]:
#Coverting the string to lower case and iterating through the words and appending them to the list if the word is not a for X_test
X_test = X_test.str.lower()

lst = []
for phase_word in X_test:
    lst.append(" ".join([re.sub('[0-9\W_]', '', word) for word in phase_word.split() if not word in sw]))
X_test = lst

In [15]:
#iterating through the word list for each sentence and assigning to tokens
for i in range(len(X)):
    X[i] = word_tokenize(X[i])

In [16]:
#iterating through the word list for each sentence and assigning to tokens
for i in range(len(X_test)):
    X_test[i] = word_tokenize(X_test[i])

In [17]:
X_train, X_val, y_train, y_val = train_test_split(np.array(X), y, test_size=0.2) 

/home/sanju/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [18]:
f = open('glove.6B.50d.txt')

embeddings_index = {}
cnt = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float')
    embeddings_index[word] = coefs
f.close()

In [19]:
def embedding_output(X):
  max_len=50
  emb_dim=50
  embedding_out = np.zeros((len(X), max_len, emb_dim))


  for ix in range(len(X)):
    for ij in range(len(X[ix])):
      try:
        embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
      except: 
        embedding_out[ix][ij] = np.zeros((50, ))

  return embedding_out

In [20]:
#assigning embedding matrix for training and validation sets
embedding_matrix_train = embedding_output(X_train)
embedding_matrix_val = embedding_output(X_val)
embedding_matrix_test=embedding_output(X_test)

In [21]:
embedding_matrix_train.shape

(21984, 50, 50)

In [22]:
#creating the model for Tweet sentiment extraction and compiling it
model = Sequential()
model.add(LSTM(64, input_shape=(50, 50)))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
prediction=model.predict
hist = model.fit(embedding_matrix_train, y_train, epochs=15, validation_data=(embedding_matrix_val, y_val))


Epoch 1/15
687/687 [==============================] - 7s 11ms/step - loss: 0.9695 - accuracy: 0.5225 - val_loss: 0.8636 - val_accuracy: 0.6112
Epoch 2/15
687/687 [==============================] - 7s 10ms/step - loss: 0.8516 - accuracy: 0.6303 - val_loss: 0.8105 - val_accuracy: 0.6470
Epoch 3/15
687/687 [==============================] - 7s 10ms/step - loss: 0.8000 - accuracy: 0.6581 - val_loss: 0.7765 - val_accuracy: 0.6659
Epoch 4/15
687/687 [==============================] - 7s 10ms/step - loss: 0.7712 - accuracy: 0.6779 - val_loss: 0.7916 - val_accuracy: 0.6723
Epoch 5/15
687/687 [==============================] - 7s 10ms/step - loss: 0.7535 - accuracy: 0.6856 - val_loss: 0.7596 - val_accuracy: 0.6830
Epoch 6/15
687/687 [==============================] - 7s 10ms/step - loss: 0.7348 - accuracy: 0.6957 - val_loss: 0.7663 - val_accuracy: 0.6767
Epoch 7/15
687/687 [==============================] - 7s 10ms/step - loss: 0.7195 - accuracy: 0.7084 - val_loss: 0.7514 - val_accuracy: 0.6750

In [23]:
model.predict(embedding_matrix_val[0:10])

array([[0.8168142 , 0.14753218, 0.03565368],
       [0.8641808 , 0.11123656, 0.02458262],
       [0.05906102, 0.88458735, 0.05635171],
       [0.96138555, 0.03174026, 0.00687418],
       [0.50477886, 0.37041605, 0.12480503],
       [0.11619938, 0.5580887 , 0.32571188],
       [0.95458037, 0.03714102, 0.00827853],
       [0.01457619, 0.05570506, 0.9297188 ],
       [0.01069314, 0.03520695, 0.95409995],
       [0.953515  , 0.03834991, 0.0081351 ]], dtype=float32)

In [24]:
y_val[0:10]

array([0, 0, 1, 0, 0, 1, 0, 2, 2, 0])

In [25]:
# Define metrics
y_pred = model.predict(embedding_matrix_test)
pred = []
for i in range(len(X_test)):
   pred.append(np.argmax(y_pred[i]))

In [26]:
pred = np.array(pred)

In [27]:
#calculating the f1 score for the model
f1_score(y_test, pred, average='macro')

0.6672028878138687

In [28]:
#calculating the recall score for the model
recall_score(y_test, pred, average='macro')

0.670677947015209

In [34]:
accuracy_score(y_test, pred)

0.6618562535370685

In [29]:
#calculating the precision score for the model
precision_score(y_test, pred, average="macro")

0.6725857862310235

In [30]:
#calculating the accuracy score for the model
accuracy_score(y_test, pred)

0.6618562535370685

In [31]:
#Display the architecture of the model
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 29,635
Trainable params: 29,635
Non-trainable params: 0
_________________________________________________________________
